In [2]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
from bs4 import BeautifulSoup
import requests

In [3]:
pokemon = pd.DataFrame(columns = ["name", "moves"], index = range(1, 1026))
pokemon

,name,moves
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN
5,NaN,NaN
...,...,...
1021,NaN,NaN
1022,NaN,NaN
1023,NaN,NaN
1024,NaN,NaN


In [6]:
pokemon = pd.read_csv("../pokemon_moves.csv", index_col = 0)

In [4]:
# Creating a function for finding stats of a pokemon 
def scrap_pokemon_moves(pokemon_name):
    pokemon = pokemon_name.replace(" ", "").lower()
    try:
        response = requests.get(f"https://www.serebii.net/pokedex-sv/{pokemon}/")
        response.raise_for_status
        text = response.text
    except requests.exceptions.RequestException as e:
        print(f"Request error {e} occured")
    soup = BeautifulSoup(text, "lxml")
    # get moves
    moves = []
    moves_data = soup.find_all("td", class_="fooinfo")
    for data in moves_data:
        if data.find("a") != None:
            link = data.find("a").get("href")
            if link != None:
                if "attackdex" in link:
                    move_data = data.find("a").text
                    if "TM" in move_data:
                        None
                    else:
                        moves.append(move_data)
    return {"name": pokemon_name, "moves": moves}

In [7]:
# Bulbasaur evolution line
bulbasaur_data = scrap_pokemon_moves("Bulbasaur")
pokemon.loc[1] = bulbasaur_data
ivysaur_data = scrap_pokemon_moves("Ivysaur")
pokemon.loc[2] = ivysaur_data
venusaur_data = scrap_pokemon_moves("Venusaur")
pokemon.loc[3] = venusaur_data

In [39]:
# Charmander line 
charmander_data = scrap_pokemon_moves("Charmander")
pokemon.loc[4] = charmander_data
charmeleon_data = scrap_pokemon_moves("Charmeleon")
pokemon.loc[5] = charmeleon_data
charizard_data = scrap_pokemon_moves("Charizard")
pokemon.loc[6] = charizard_data

In [40]:
# Squirtle line 
squirtle_data = scrap_pokemon_moves("Squirtle")
pokemon.loc[7] = squirtle_data
wartortle_data = scrap_pokemon_moves("Wartortle")
pokemon.loc[8] = wartortle_data
blastoise_data = scrap_pokemon_moves("Blastoise")
pokemon.loc[9] = blastoise_data

In [41]:
# Caterpie line 
caterpie_data = scrap_pokemon_moves("Caterpie")
pokemon.loc[10] = caterpie_data
metapod_data = scrap_pokemon_moves("Metapod")
pokemon.loc[11] = metapod_data
butterfree_data = scrap_pokemon_moves("Butterfree")
pokemon.loc[12] = butterfree_data

In [42]:
# Weedle line 
weedle_data = scrap_pokemon_moves("Weedle")
pokemon.loc[13] = weedle_data
kakuna_data = scrap_pokemon_moves("Kakuna")
pokemon.loc[14] = kakuna_data
beedrill_data = scrap_pokemon_moves("Beedrill")
pokemon.loc[15] = beedrill_data

In [43]:
#pidgey line
pidgey_data = scrap_pokemon_moves("Pidgey")
pokemon.loc[16] = pidgey_data
pidgeotto_data = scrap_pokemon_moves("Pidgeotto")
pokemon.loc[17] = pidgeotto_data
pidgeot_data = scrap_pokemon_moves("Pidgeot")
pokemon.loc[18] = pidgeot_data

In [44]:
# Rattata line
rattata_data = scrap_pokemon_moves("Rattata")
pokemon.loc[19] = rattata_data
raticate_data = scrap_pokemon_moves("Raticate")
pokemon.loc[20] = raticate_data

In [45]:
#Spearow line
spearow_data = scrap_pokemon_moves("Spearow")
pokemon.loc[21] = spearow_data
fearow_data = scrap_pokemon_moves("Fearow")
pokemon.loc[22] = fearow_data

In [46]:
# Ekans line
ekans_data = scrap_pokemon_moves("Ekans")
pokemon.loc[23] = ekans_data
arbok_data = scrap_pokemon_moves("Arbok")
pokemon.loc[24] = arbok_data

In [47]:
#Pikachu line
pikachu_data = scrap_pokemon_moves("Pikachu")
pokemon.loc[25] = pikachu_data
raichu_data = scrap_pokemon_moves("Raichu")
pokemon.loc[26] = raichu_data


In [48]:
#Sandshrew line
sandshrew_data = scrap_pokemon_moves("Sandshrew")
pokemon.loc[27] = sandshrew_data
sandslash_data = scrap_pokemon_moves("Sandslash")
pokemon.loc[28] = sandslash_data

In [49]:
# nidoran female line
nidoranf_data = scrap_pokemon_moves("Nidoran♀")
pokemon.loc[29] = nidoranf_data
nidorina_data = scrap_pokemon_moves("Nidorina")
pokemon.loc[30] = nidorina_data
nidoqueen_data = scrap_pokemon_moves("Nidoqueen")
pokemon.loc[31] = nidoqueen_data

In [50]:
# Nidoran male line
nidoranm_data = scrap_pokemon_moves("Nidoran♂")
pokemon.loc[32] = nidoranm_data
nidorino_data = scrap_pokemon_moves("Nidorino")
pokemon.loc[33] = nidorino_data
nidoking_data = scrap_pokemon_moves("Nidoking")
pokemon.loc[34] = nidoking_data

In [51]:
#Clefairy line 
clefairy_data = scrap_pokemon_moves("Clefairy")
pokemon.loc[35] = clefairy_data
clefable_data = scrap_pokemon_moves("Clefable")
pokemon.loc[36] = clefable_data

In [52]:
# Vulpix line
vulpix_data = scrap_pokemon_moves("Vulpix")
pokemon.loc[37] = vulpix_data
ninetails_data = scrap_pokemon_moves("Ninetales")
pokemon.loc[38] = ninetails_data

In [53]:
# Jigglypuff line
jigglypuff_data = scrap_pokemon_moves("Jigglypuff")
pokemon.loc[39] = jigglypuff_data
wigglytuff_data = scrap_pokemon_moves("Wigglytuff")
pokemon.loc[40] = wigglytuff_data

In [54]:
#Zubat line
zubat_data = scrap_pokemon_moves("Zubat")
pokemon.loc[41] = zubat_data
golbat_data = scrap_pokemon_moves("Golbat")
pokemon.loc[42] = golbat_data

In [55]:
#Oddish line
oddish_data = scrap_pokemon_moves("Oddish")
pokemon.loc[43] = oddish_data
gloom_data = scrap_pokemon_moves("Gloom")
pokemon.loc[44] = gloom_data
vileplume_data = scrap_pokemon_moves("Vileplume")
pokemon.loc[45] = vileplume_data

In [56]:
# paras line
paras_data = scrap_pokemon_moves("Paras")
pokemon.loc[46] = paras_data
parasect_data = scrap_pokemon_moves("Parasect")
pokemon.loc[47] = parasect_data

In [57]:
# Venonat line
venonat_data = scrap_pokemon_moves("Venonat")
pokemon.loc[48] = venonat_data
venomoth_data = scrap_pokemon_moves("Venomoth")
pokemon.loc[49] = venomoth_data

In [58]:
# diglett line
diglett_data = scrap_pokemon_moves("Diglett")
pokemon.loc[50] = diglett_data
dugtrio_data = scrap_pokemon_moves("Dugtrio")
pokemon.loc[51] = dugtrio_data

In [59]:
# meowth line
meowth_data = scrap_pokemon_moves("Meowth")
pokemon.loc[52] = meowth_data
persian_data = scrap_pokemon_moves("Persian")
pokemon.loc[53] = persian_data 

In [60]:
# Psyduck line
psyduck_data = scrap_pokemon_moves("Psyduck")
pokemon.loc[54] = psyduck_data
golduck_data = scrap_pokemon_moves("Golduck")
pokemon.loc[55] = golduck_data

In [61]:
# Mankey line 
mankey_data = scrap_pokemon_moves("Mankey")
pokemon.loc[56] = mankey_data
primeape_data = scrap_pokemon_moves("Primeape")
pokemon.loc[57] = primeape_data

In [62]:
# Growlithe line
growlithe_data = scrap_pokemon_moves("Growlithe")
pokemon.loc[58] = growlithe_data
arcanine_data = scrap_pokemon_moves("Arcanine")
pokemon.loc[59] = arcanine_data

In [63]:
# poliwag line
poliwag_data = scrap_pokemon_moves("Poliwag")
pokemon.loc[60] = poliwag_data
poliwhirl_data = scrap_pokemon_moves("Poliwhirl")
pokemon.loc[61] = poliwhirl_data
poliwrath_data = scrap_pokemon_moves("Poliwrath")
pokemon.loc[62] = poliwrath_data

In [64]:
# Abra line 
abra_data = scrap_pokemon_moves("Abra")
pokemon.loc[63] = abra_data
kadabra_data = scrap_pokemon_moves("Kadabra")
pokemon.loc[64] = kadabra_data
alakazam_data = scrap_pokemon_moves("Alakazam")
pokemon.loc[65] = alakazam_data

In [65]:
# Machop line
machop_data = scrap_pokemon_moves("Machop")
pokemon.loc[66] = machop_data
machoke_data = scrap_pokemon_moves("Machoke")
pokemon.loc[67] = machoke_data
machamp_data = scrap_pokemon_moves("Machamp")
pokemon.loc[68] = machamp_data

In [66]:
# bellsprout line 
bellsprout_data = scrap_pokemon_moves("Bellsprout")
pokemon.loc[69] = bellsprout_data
weepinbell_data = scrap_pokemon_moves("Weepinbell")
pokemon.loc[70] = weepinbell_data
victreebel_data = scrap_pokemon_moves("Victreebel")
pokemon.loc[71] = victreebel_data

In [67]:
# tentacool line
tentacool_data = scrap_pokemon_moves("Tentacool")
pokemon.loc[72] = tentacool_data
tentacruel_data = scrap_pokemon_moves("Tentacruel")
pokemon.loc[73] = tentacruel_data

In [68]:
# geodude line 
geodude_data = scrap_pokemon_moves("Geodude")
pokemon.loc[74] = geodude_data
graveler_data = scrap_pokemon_moves("Graveler")
pokemon.loc[75] = graveler_data
golem_data = scrap_pokemon_moves("Golem")
pokemon.loc[76] = golem_data

In [69]:
# ponyta line 
ponyta_data = scrap_pokemon_moves("Ponyta")
pokemon.loc[77] = ponyta_data
rapidash_data = scrap_pokemon_moves("Rapidash")
pokemon.loc[78] = rapidash_data

In [70]:
# Slowpoke line 
slowpoke_data = scrap_pokemon_moves("Slowpoke")
pokemon.loc[79] = slowpoke_data
slowbro_data = scrap_pokemon_moves("Slowbro")
pokemon.loc[80] = slowbro_data

In [72]:
# magnemite line
magnemite_data = scrap_pokemon_moves("Magnemite")
pokemon.loc[81] = magnemite_data
magneton_data = scrap_pokemon_moves("Magneton")
pokemon.loc[82] = magneton_data

In [74]:
# Farfetch'd
farfetch_data = scrap_pokemon_moves("Farfetch'd")
pokemon.loc[83] = farfetch_data

In [76]:
# doduo line
doduo_data = scrap_pokemon_moves("Doduo")
pokemon.loc[84] = doduo_data
dodrio_data = scrap_pokemon_moves("Dodrio")
pokemon.loc[85] = dodrio_data

In [77]:
# Seel line 
seel_data = scrap_pokemon_moves("Seel")
pokemon.loc[86] = seel_data
dewgong_data = scrap_pokemon_moves("Dewgong")
pokemon.loc[87] = dewgong_data

In [78]:
# grimer line 
grimer_data = scrap_pokemon_moves("Grimer")
pokemon.loc[88] = grimer_data
muk_data = scrap_pokemon_moves("Muk")
pokemon.loc[89] = muk_data

In [79]:
# Shellder line 
shellder_data = scrap_pokemon_moves("Shellder")
pokemon.loc[90] = shellder_data
cloyster_data = scrap_pokemon_moves("Cloyster")
pokemon.loc[91] = cloyster_data

In [80]:
# Gastly line
gastly_data = scrap_pokemon_moves("Gastly")
pokemon.loc[92] = gastly_data
haunter_data = scrap_pokemon_moves("Haunter")
pokemon.loc[93] = haunter_data
gengar_data = scrap_pokemon_moves("Gengar")
pokemon.loc[94] = gengar_data

In [81]:
# onix line 
onix_data = scrap_pokemon_moves("Onix")
pokemon.loc[95] = onix_data

In [82]:
# drowzee line 
drowzee_data = scrap_pokemon_moves("Drowzee")
pokemon.loc[96] = drowzee_data
hypno_data = scrap_pokemon_moves("Hypno")
pokemon.loc[97] = hypno_data

In [83]:
# Krabby line 
krabby_data = scrap_pokemon_moves("Krabby")
pokemon.loc[98] = krabby_data
kingler_data = scrap_pokemon_moves("Kingler")
pokemon.loc[99] = kingler_data

In [84]:
# voltorb line 
voltorb_data = scrap_pokemon_moves("Voltorb")
pokemon.loc[100] = voltorb_data
electrode_data = scrap_pokemon_moves("Electrode")
pokemon.loc[101] = electrode_data

In [85]:
# Exeggcute line 
exeggcute_data = scrap_pokemon_moves("Exeggcute")
pokemon.loc[102] = exeggcute_data
exeggutor_data = scrap_pokemon_moves("Exeggutor")
pokemon.loc[103] = exeggutor_data

In [86]:
# cubone line 
cubone_data = scrap_pokemon_moves("Cubone")
pokemon.loc[104] = cubone_data
marowak_data = scrap_pokemon_moves("Marowak")
pokemon.loc[105] = marowak_data

In [87]:
# hitmonlee line  and lickitung
hitmonlee_data = scrap_pokemon_moves("Hitmonlee")
pokemon.loc[106] = hitmonlee_data
hitmonchan_data = scrap_pokemon_moves("Hitmonchan")
pokemon.loc[107] = hitmonchan_data
lickitung_data = scrap_pokemon_moves("Lickitung")
pokemon.loc[108] = lickitung_data

In [88]:
# koffing
koffing_data = scrap_pokemon_moves("Koffing")
pokemon.loc[109] = koffing_data
weezing_data = scrap_pokemon_moves("Weezing")
pokemon.loc[110] = weezing_data

In [89]:
# Rhyhorn line
rhyhorn_data = scrap_pokemon_moves("Rhyhorn")
pokemon.loc[111] = rhyhorn_data
rhydon_data = scrap_pokemon_moves("Rhydon")
pokemon.loc[112] = rhydon_data

In [90]:
# Chansey line and tangela line
chansey_data = scrap_pokemon_moves("Chansey")
pokemon.loc[113] = chansey_data
tangela_data = scrap_pokemon_moves("Tangela")
pokemon.loc[114] = tangela_data

In [91]:
#Kangaskhan line
kangaskhan_data = scrap_pokemon_moves("Kangaskhan")
pokemon.loc[115] = kangaskhan_data

In [92]:
# horsea line 
horsea_data = scrap_pokemon_moves("Horsea")
pokemon.loc[116] = horsea_data
seadra_data = scrap_pokemon_moves("Seadra")
pokemon.loc[117] = seadra_data

In [93]:
# goldeen and staryu line
goldeen_data = scrap_pokemon_moves("Goldeen")
pokemon.loc[118] = goldeen_data
seaking_data = scrap_pokemon_moves("Seaking")
pokemon.loc[119] = seaking_data
staryu_data = scrap_pokemon_moves("Staryu")
pokemon.loc[120] = staryu_data
starmie_data = scrap_pokemon_moves("Starmie")
pokemon.loc[121] = starmie_data

In [7]:
# Mr mime line 
mrmime_data = scrap_pokemon_moves("Mr._Mime")
pokemon.loc[122] = mrmime_data

In [95]:
# Scyther and jynx line 
scyther_data = scrap_pokemon_moves("Scyther")
pokemon.loc[123] = scyther_data
jynx_data = scrap_pokemon_moves("Jynx")
pokemon.loc[124] = jynx_data

In [96]:
# Electabuzz and Magmar and pinsir
electabuzz_data = scrap_pokemon_moves("Electabuzz")
pokemon.loc[125] = electabuzz_data
magmar_data = scrap_pokemon_moves("Magmar")
pokemon.loc[126] = magmar_data
pinsir_data = scrap_pokemon_moves("Pinsir")
pokemon.loc[127] = pinsir_data

In [97]:
# Tauros and magicarp line
tauros_data = scrap_pokemon_moves("Tauros")
pokemon.loc[128] = tauros_data
magikarp_data = scrap_pokemon_moves("Magikarp")
pokemon.loc[129] = magikarp_data
gyarados_data = scrap_pokemon_moves("Gyarados")
pokemon.loc[130] = gyarados_data

In [98]:
# Lapras, ditto
lapras_data = scrap_pokemon_moves("Lapras")
pokemon.loc[131] = lapras_data
ditto_data = scrap_pokemon_moves("Ditto")
pokemon.loc[132] = ditto_data

In [99]:
# Eevee line
eevee_data = scrap_pokemon_moves("Eevee")
pokemon.loc[133] = eevee_data
vaporeon_data = scrap_pokemon_moves("Vaporeon")
pokemon.loc[134] = vaporeon_data
jolteon_data = scrap_pokemon_moves("Jolteon")
pokemon.loc[135] = jolteon_data
flareon_data = scrap_pokemon_moves("Flareon")
pokemon.loc[136] = flareon_data

In [100]:
# porygon and omanyte and kabuto line 
porygon_data = scrap_pokemon_moves("Porygon")
pokemon.loc[137] = porygon_data
omanyte_data = scrap_pokemon_moves("Omanyte")
pokemon.loc[138] = omanyte_data
omastar_data = scrap_pokemon_moves("Omastar")
pokemon.loc[139] = omastar_data
kabuto_data = scrap_pokemon_moves("Kabuto")
pokemon.loc[140] = kabuto_data
kabutops_data = scrap_pokemon_moves("Kabutops")
pokemon.loc[141] = kabutops_data

In [101]:
# Aerodactyl line and snorlax line
aerodactyl_data = scrap_pokemon_moves("Aerodactyl")
pokemon.loc[142] = aerodactyl_data
snorlax_data = scrap_pokemon_moves("Snorlax")
pokemon.loc[143] = snorlax_data

In [102]:
# Legendary birds
articuno_data = scrap_pokemon_moves("Articuno")
pokemon.loc[144] = articuno_data
zapdos_data = scrap_pokemon_moves("Zapdos") 
pokemon.loc[145] = zapdos_data
moltres_data = scrap_pokemon_moves("Moltres")
pokemon.loc[146] = moltres_data

In [103]:
# Dratini line 
dratini_data = scrap_pokemon_moves("Dratini")
pokemon.loc[147] = dratini_data
dragonair_data = scrap_pokemon_moves("Dragonair")
pokemon.loc[148] = dragonair_data
dragonite_data = scrap_pokemon_moves("Dragonite")
pokemon.loc[149] = dragonite_data

In [104]:
# Mew and mewtwo 
mewtwo_data = scrap_pokemon_moves("Mewtwo")
pokemon.loc[150] = mewtwo_data
mew_data = scrap_pokemon_moves("Mew")
pokemon.loc[151] = mew_data

In [105]:
# Chikorita line 
chikorita = scrap_pokemon_moves("Chikorita")
pokemon.loc[152] = chikorita
bayleef = scrap_pokemon_moves("Bayleef")
pokemon.loc[153] = bayleef
meganium = scrap_pokemon_moves("Meganium")
pokemon.loc[154] = meganium

In [106]:
# Cyndaquil line
cyndaquil = scrap_pokemon_moves("Cyndaquil")
pokemon.loc[155] = cyndaquil
quilava = scrap_pokemon_moves("Quilava")
pokemon.loc[156] = quilava
typhlosion = scrap_pokemon_moves("Typhlosion")
pokemon.loc[157] = typhlosion

In [107]:
# totodile line 
totodile = scrap_pokemon_moves("Totodile")
pokemon.loc[158] = totodile
croconaw = scrap_pokemon_moves("Croconaw")
pokemon.loc[159] = croconaw
feraligatr = scrap_pokemon_moves("Feraligatr")
pokemon.loc[160] = feraligatr

In [108]:
# sentret line 
sentret = scrap_pokemon_moves("Sentret")
pokemon.loc[161] = sentret
furret = scrap_pokemon_moves("Furret")
pokemon.loc[162] = furret

In [109]:
# hoothoot line 
hoothoot = scrap_pokemon_moves("Hoothoot")
pokemon.loc[163] = hoothoot
noctowl = scrap_pokemon_moves("Noctowl")
pokemon.loc[164] = noctowl

In [110]:
# ledyba and spinarak line 
ledyba = scrap_pokemon_moves("Ledyba")
pokemon.loc[165] = ledyba
ledian = scrap_pokemon_moves("Ledian")
pokemon.loc[166] = ledian
spinarak = scrap_pokemon_moves("Spinarak")
pokemon.loc[167] = spinarak
ariados = scrap_pokemon_moves("Ariados")
pokemon.loc[168] = ariados

In [111]:
# Crobat and chichou line 
crobat = scrap_pokemon_moves("Crobat")
pokemon.loc[169] = crobat
chinchou = scrap_pokemon_moves("Chinchou")
pokemon.loc[170] = chinchou
lanturn = scrap_pokemon_moves("Lanturn")
pokemon.loc[171] = lanturn

In [112]:
# baby pokemon
pichu = scrap_pokemon_moves("Pichu")
pokemon.loc[172] = pichu
cleffa = scrap_pokemon_moves("Cleffa")
pokemon.loc[173] = cleffa
igglybuff = scrap_pokemon_moves("Igglybuff")
pokemon.loc[174] = igglybuff
togepi = scrap_pokemon_moves("Togepi")
pokemon.loc[175] = togepi 
togetic = scrap_pokemon_moves("Togetic")
pokemon.loc[176] = togetic

In [113]:
# natu line 
natu = scrap_pokemon_moves("Natu")
pokemon.loc[177] = natu
xatu = scrap_pokemon_moves("Xatu")
pokemon.loc[178] = xatu

In [114]:
# mareep line
mareep = scrap_pokemon_moves("Mareep")
pokemon.loc[179] = mareep
flaaffy = scrap_pokemon_moves("Flaaffy")
pokemon.loc[180] = flaaffy
ampharos = scrap_pokemon_moves("Ampharos")
pokemon.loc[181] = ampharos

In [115]:
# bellossom and maril
bellossom = scrap_pokemon_moves("Bellossom")
pokemon.loc[182] = bellossom
marill = scrap_pokemon_moves("Marill")
pokemon.loc[183] = marill
azumarill = scrap_pokemon_moves("Azumarill")
pokemon.loc[184] = azumarill
sudowoodo = scrap_pokemon_moves("Sudowoodo")
pokemon.loc[185] = sudowoodo
politoed = scrap_pokemon_moves("Politoed")
pokemon.loc[186] = politoed

In [116]:
# hopip line
hoppip = scrap_pokemon_moves("Hoppip")
pokemon.loc[187] = hoppip
skiploom = scrap_pokemon_moves("Skiploom")
pokemon.loc[188] = skiploom
jumpluff = scrap_pokemon_moves("Jumpluff")
pokemon.loc[189] = jumpluff

In [117]:
# Aipom, sunkern
aipom = scrap_pokemon_moves("Aipom")
pokemon.loc[190] = aipom
sunkern = scrap_pokemon_moves("Sunkern")
pokemon.loc[191] = sunkern
sunflora = scrap_pokemon_moves("Sunflora")
pokemon.loc[192] = sunflora
yanma = scrap_pokemon_moves("Yanma")
pokemon.loc[193] = yanma

In [118]:
# wooper
wooper = scrap_pokemon_moves("Wooper")
pokemon.loc[194] = wooper
quagsire = scrap_pokemon_moves("Quagsire")
pokemon.loc[195] = quagsire

In [119]:
# Eevvelutions
espeon = scrap_pokemon_moves("Espeon")
pokemon.loc[196] = espeon
umbreon = scrap_pokemon_moves("Umbreon")
pokemon.loc[197] = umbreon
murkrow = scrap_pokemon_moves("Murkrow")
pokemon.loc[198] = murkrow

In [18]:
# Slowking
slowking = scrap_pokemon_moves("Slowking")
pokemon.loc[199] = slowking
misdreavus = scrap_pokemon_moves("Misdreavus")
pokemon.loc[200] = misdreavus
unown = scrap_pokemon_moves("Unown")
pokemon.loc[201] = unown
wobbuffet = scrap_pokemon_moves("Wobbuffet")
pokemon.loc[202] = wobbuffet
girafarig = scrap_pokemon_moves("Girafarig")
pokemon.loc[203] = girafarig

In [121]:
# Pineco line
pineco = scrap_pokemon_moves("Pineco")
pokemon.loc[204] = pineco
forretress = scrap_pokemon_moves("Forretress")
pokemon.loc[205] = forretress
pokemon.loc[206] = scrap_pokemon_moves("Dunsparce")

In [122]:
# gligar, sleelix
pokemon.loc[207] = scrap_pokemon_moves("Gligar")
steelix = scrap_pokemon_moves("Steelix")
pokemon.loc[208] = steelix
pokemon.loc[209] = scrap_pokemon_moves("Snubbull")
pokemon.loc[210] = scrap_pokemon_moves("Granbull")

In [123]:
# Qilfish and scizor
pokemon.loc[211] = scrap_pokemon_moves("Qwilfish")
scizor = scrap_pokemon_moves("Scizor")
pokemon.loc[212] = scizor
pokemon.loc[213] = scrap_pokemon_moves("Shuckle")

In [124]:
# Herracross
heracross = scrap_pokemon_moves("Heracross")
pokemon.loc[214] = heracross
pokemon.loc[215] = scrap_pokemon_moves("Sneasel")
pokemon.loc[216] = scrap_pokemon_moves("Teddiursa")
pokemon.loc[217] = scrap_pokemon_moves("Ursaring")
pokemon.loc[218] = scrap_pokemon_moves("Slugma")
pokemon.loc[219] = scrap_pokemon_moves("Magcargo")
pokemon.loc[220] = scrap_pokemon_moves("Swinub")
pokemon.loc[221] = scrap_pokemon_moves("Piloswine")

In [125]:
# corsola - houndoom
corsola = scrap_pokemon_moves("Corsola")
pokemon.loc[222] = corsola
pokemon.loc[223] = scrap_pokemon_moves("Remoraid")
pokemon.loc[224] = scrap_pokemon_moves("Octillery")
pokemon.loc[225] = scrap_pokemon_moves("Delibird")
pokemon.loc[226] = scrap_pokemon_moves("Mantine")
pokemon.loc[227] = scrap_pokemon_moves("Skarmory")
pokemon.loc[228] = scrap_pokemon_moves("Houndour")
houndoom = scrap_pokemon_moves("Houndoom")
pokemon.loc[229] = houndoom

In [ ]:
# kingdra
pokemon.loc[230] = scrap_pokemon_moves("Kingdra")
pokemon.loc[231] = scrap_pokemon_moves("Phanpy")
pokemon.loc[232] = scrap_pokemon_moves("Donphan")
pokemon.loc[233] = scrap_pokemon_moves("Porygon2")
pokemon.loc[234] = scrap_pokemon_moves("Stantler")
pokemon.loc[235] = scrap_pokemon_moves("Smeargle")
pokemon.loc[236] = scrap_pokemon_moves("Tyrogue")
pokemon.loc[237] = scrap_pokemon_moves("Hitmontop")
pokemon.loc[238] = scrap_pokemon_moves("Smoochum")
pokemon.loc[239] = scrap_pokemon_moves("Elekid")
pokemon.loc[240] = scrap_pokemon_moves("Magby")

In [127]:
# Milktank
pokemon.loc[241] = scrap_pokemon_moves("Miltank")
pokemon.loc[242] = scrap_pokemon_moves("Blissey")

In [128]:
#Legendary beasts
raikou = scrap_pokemon_moves("Raikou")
pokemon.loc[243] = raikou
entei = scrap_pokemon_moves("Entei")
pokemon.loc[244] = entei
suicune = scrap_pokemon_moves("Suicune")
pokemon.loc[245] = suicune

In [129]:
# larvitar line 
pokemon.loc[246] = scrap_pokemon_moves("Larvitar")
pokemon.loc[247] = scrap_pokemon_moves("Pupitar")
tyranitar = scrap_pokemon_moves("Tyranitar")
pokemon.loc[248] = tyranitar
pokemon.loc[249] = scrap_pokemon_moves("Lugia")
pokemon.loc[250] = scrap_pokemon_moves("Ho-Oh")
pokemon.loc[251] = scrap_pokemon_moves("Celebi")

In [130]:
# treeko line 
pokemon.loc[252] = scrap_pokemon_moves("Treecko")
pokemon.loc[253] = scrap_pokemon_moves("Grovyle")
sceptile = scrap_pokemon_moves("Sceptile")
pokemon.loc[254] = sceptile

In [131]:
# torchic line
pokemon.loc[255] = scrap_pokemon_moves("Torchic")
pokemon.loc[256] = scrap_pokemon_moves("Combusken")
blaziken = scrap_pokemon_moves("Blaziken")
pokemon.loc[257] = blaziken

In [132]:
# mudkip line
pokemon.loc[258] = scrap_pokemon_moves("Mudkip")
pokemon.loc[259] = scrap_pokemon_moves("Marshtomp")
swampert = scrap_pokemon_moves("Swampert")
pokemon.loc[260] = swampert

In [133]:
# poochyena
pokemon.loc[261] = scrap_pokemon_moves("Poochyena")
pokemon.loc[262] = scrap_pokemon_moves("Mightyena")
pokemon.loc[263] = scrap_pokemon_moves("Zigzagoon")
pokemon.loc[264] = scrap_pokemon_moves("Linoone")

In [134]:
# Wurmple line and lotad and seedot
pokemon.loc[265] = scrap_pokemon_moves("Wurmple")
pokemon.loc[266] = scrap_pokemon_moves("Silcoon")
pokemon.loc[267] = scrap_pokemon_moves("Beautifly")
pokemon.loc[268] = scrap_pokemon_moves("Cascoon")
pokemon.loc[269] = scrap_pokemon_moves("Dustox")
pokemon.loc[270] = scrap_pokemon_moves("Lotad")
pokemon.loc[271] = scrap_pokemon_moves("Lombre")
pokemon.loc[272] = scrap_pokemon_moves("Ludicolo")
pokemon.loc[273] = scrap_pokemon_moves("Seedot")
pokemon.loc[274] = scrap_pokemon_moves("Nuzleaf")

In [6]:
# shiftry Tailow line and wingull
shiftry = scrap_pokemon_moves("Shiftry")
pokemon.loc[275] = shiftry
pokemon.loc[276] = scrap_pokemon_moves("Taillow")
pokemon.loc[277] = scrap_pokemon_moves("Swellow")
pokemon.loc[278] = scrap_pokemon_moves("Wingull")
pokemon.loc[279] = scrap_pokemon_moves("Pelipper")

In [7]:
# ralts line 
pokemon.loc[280] = scrap_pokemon_moves("Ralts")
pokemon.loc[281] = scrap_pokemon_moves("Kirlia")
gardevoir = scrap_pokemon_moves("Gardevoir")
pokemon.loc[282] = gardevoir

In [8]:
# surskit line and shroomish and slakoth
pokemon.loc[283] = scrap_pokemon_moves("Surskit")
pokemon.loc[284] = scrap_pokemon_moves("Masquerain")
pokemon.loc[285] = scrap_pokemon_moves("Shroomish")
pokemon.loc[286] = scrap_pokemon_moves("Breloom")
pokemon.loc[287] = scrap_pokemon_moves("Slakoth")
pokemon.loc[288] = scrap_pokemon_moves("Vigoroth")
pokemon.loc[289] = scrap_pokemon_moves("Slaking")

In [9]:
# nincada line and whismur 
pokemon.loc[290] = scrap_pokemon_moves("Nincada")
pokemon.loc[291] = scrap_pokemon_moves("Ninjask")
pokemon.loc[292] = scrap_pokemon_moves("Shedinja")
pokemon.loc[293] = scrap_pokemon_moves("Whismur")
pokemon.loc[294] = scrap_pokemon_moves("Loudred")
pokemon.loc[295] = scrap_pokemon_moves("Exploud")

In [10]:
# makuhita and skitty
pokemon.loc[296] = scrap_pokemon_moves("Makuhita")
pokemon.loc[297] = scrap_pokemon_moves("Hariyama")
pokemon.loc[298] = scrap_pokemon_moves("Azurill")
pokemon.loc[299] = scrap_pokemon_moves("Nosepass")
pokemon.loc[300] = scrap_pokemon_moves("Skitty")
pokemon.loc[301] = scrap_pokemon_moves("Delcatty")

In [11]:
# sableye
sableye = scrap_pokemon_moves("Sableye")
pokemon.loc[302] = sableye
mawile = scrap_pokemon_moves("Mawile")
pokemon.loc[303] = mawile

In [12]:
# Aron line'
pokemon.loc[304] = scrap_pokemon_moves("Aron")
pokemon.loc[305] = scrap_pokemon_moves("Lairon")
aggron = scrap_pokemon_moves("Aggron")
pokemon.loc[306] = aggron

In [13]:
# meditate and electric
pokemon.loc[307] = scrap_pokemon_moves("Meditite")
medicham = scrap_pokemon_moves("Medicham")
pokemon.loc[308] = medicham
pokemon.loc[309] = scrap_pokemon_moves("Electrike")
manectric = scrap_pokemon_moves("Manectric")
pokemon.loc[310] = manectric

In [14]:
# plusle
pokemon.loc[311] = scrap_pokemon_moves("Plusle")
pokemon.loc[312] = scrap_pokemon_moves("Minun")
pokemon.loc[313] = scrap_pokemon_moves("Volbeat")
pokemon.loc[314] = scrap_pokemon_moves("Illumise")
pokemon.loc[315] = scrap_pokemon_moves("Roselia")
pokemon.loc[316] = scrap_pokemon_moves("Gulpin")
pokemon.loc[317] = scrap_pokemon_moves("Swalot")

In [15]:
# carvanah line
pokemon.loc[318] = scrap_pokemon_moves("Carvanha")
sharpedo = scrap_pokemon_moves("Sharpedo")
pokemon.loc[319] = sharpedo
pokemon.loc[320] = scrap_pokemon_moves("Wailmer")
pokemon.loc[321] = scrap_pokemon_moves("Wailord")

In [16]:
# numel
pokemon.loc[322] = scrap_pokemon_moves("Numel")
camerupt = scrap_pokemon_moves("Camerupt")
pokemon.loc[323] = camerupt
pokemon.loc[324] = scrap_pokemon_moves("Torkoal")
pokemon.loc[325] = scrap_pokemon_moves("Spoink")
pokemon.loc[326] = scrap_pokemon_moves("Grumpig")
pokemon.loc[327] = scrap_pokemon_moves("Spinda")

In [17]:
# trapinch to swablu
pokemon.loc[328] = scrap_pokemon_moves("Trapinch")
pokemon.loc[329] = scrap_pokemon_moves("Vibrava")
pokemon.loc[330] = scrap_pokemon_moves("Flygon")
pokemon.loc[331] = scrap_pokemon_moves("Cacnea")
pokemon.loc[332] = scrap_pokemon_moves("Cacturne")
pokemon.loc[333] = scrap_pokemon_moves("Swablu")
altaria = scrap_pokemon_moves("Altaria")
pokemon.loc[334] = altaria

In [18]:
# zangoose
pokemon.loc[335] = scrap_pokemon_moves("Zangoose")
pokemon.loc[336] = scrap_pokemon_moves("Seviper")
pokemon.loc[337] = scrap_pokemon_moves("Lunatone")
pokemon.loc[338] = scrap_pokemon_moves("Solrock")
pokemon.loc[339] = scrap_pokemon_moves("Barboach")
pokemon.loc[340] = scrap_pokemon_moves("Whiscash")
pokemon.loc[341] = scrap_pokemon_moves("Corphish")
pokemon.loc[342] = scrap_pokemon_moves("Crawdaunt")
pokemon.loc[343] = scrap_pokemon_moves("Baltoy")
pokemon.loc[344] = scrap_pokemon_moves("Claydol")

In [17]:
# fossils 3rd gen
pokemon.loc[345] = scrap_pokemon_moves("Lileep")
pokemon.loc[346] = scrap_pokemon_moves("Cradily")
pokemon.loc[347] = scrap_pokemon_moves("Anorith")
pokemon.loc[348] = scrap_pokemon_moves("Armaldo")
pokemon.loc[349] = scrap_pokemon_moves("Feebas")
pokemon.loc[350] = scrap_pokemon_moves("Milotic")

In [20]:
# castform
pokemon.loc[351] = scrap_pokemon_moves("Castform")

In [21]:
# kecleon
pokemon.loc[352] = scrap_pokemon_moves("Kecleon")
pokemon.loc[353] = scrap_pokemon_moves("Shuppet")
banette = scrap_pokemon_moves("Banette")
pokemon.loc[354] = banette
pokemon.loc[355] = scrap_pokemon_moves("Duskull")
pokemon.loc[356] = scrap_pokemon_moves("Dusclops")
pokemon.loc[357] = scrap_pokemon_moves("Tropius")
pokemon.loc[358] = scrap_pokemon_moves("Chimecho")

In [22]:
# absol and wynaut
absol = scrap_pokemon_moves("Absol")
pokemon.loc[359] = absol
pokemon.loc[360] = scrap_pokemon_moves("Wynaut")
pokemon.loc[361] = scrap_pokemon_moves("Snorunt")
glalie = scrap_pokemon_moves("Glalie")
pokemon.loc[362] = glalie

In [23]:
# Spheal line
pokemon.loc[363] = scrap_pokemon_moves("Spheal")
pokemon.loc[364] = scrap_pokemon_moves("Sealeo")
pokemon.loc[365] = scrap_pokemon_moves("Walrein")
pokemon.loc[366] = scrap_pokemon_moves("Clamperl")
pokemon.loc[367] = scrap_pokemon_moves("Huntail")
pokemon.loc[368] = scrap_pokemon_moves("Gorebyss")
pokemon.loc[369] = scrap_pokemon_moves("Relicanth")
pokemon.loc[370] = scrap_pokemon_moves("Luvdisc")

In [24]:
# bagon line and beldum
pokemon.loc[371] = scrap_pokemon_moves("Bagon")
pokemon.loc[372] = scrap_pokemon_moves("Shelgon")
salamence = scrap_pokemon_moves("Salamence")
pokemon.loc[373] = salamence
pokemon.loc[374] = scrap_pokemon_moves("Beldum")
pokemon.loc[375] = scrap_pokemon_moves("Metang")
metagross = scrap_pokemon_moves("Metagross")
pokemon.loc[376] = metagross

In [25]:
# Regis
pokemon.loc[377] = scrap_pokemon_moves("Regirock")
pokemon.loc[378] = scrap_pokemon_moves("Regice")
pokemon.loc[379] = scrap_pokemon_moves("Registeel")

In [26]:
# latias and latious and other legendaries 
pokemon.loc[380] = scrap_pokemon_moves("Latias")
pokemon.loc[381] = scrap_pokemon_moves("Latios")
kyogre = scrap_pokemon_moves("Kyogre")
pokemon.loc[382] = kyogre
groudon = scrap_pokemon_moves("Groudon")
pokemon.loc[383] = groudon
rayquaza = scrap_pokemon_moves("Rayquaza")
pokemon.loc[384] = rayquaza
pokemon.loc[385] = scrap_pokemon_moves("Jirachi")
pokemon.loc[386] = scrap_pokemon_moves("Deoxys")

In [27]:
# Gen 4 starters
pokemon.loc[387] = scrap_pokemon_moves("Turtwig")
pokemon.loc[388] = scrap_pokemon_moves("Grotle")
pokemon.loc[389] = scrap_pokemon_moves("Torterra")
pokemon.loc[390] = scrap_pokemon_moves("Chimchar")
pokemon.loc[391] = scrap_pokemon_moves("Monferno")
pokemon.loc[392] = scrap_pokemon_moves("Infernape")
piplup = scrap_pokemon_moves("Piplup")
pokemon.loc[393] = piplup
prinplup = scrap_pokemon_moves("Prinplup")
pokemon.loc[394] = prinplup
empoleon = scrap_pokemon_moves("Empoleon")
pokemon.loc[395] = empoleon

In [28]:
# starly and bidoof
pokemon.loc[396] = scrap_pokemon_moves("Starly")
pokemon.loc[397] = scrap_pokemon_moves("Staravia")
pokemon.loc[398] = scrap_pokemon_moves("Staraptor")
pokemon.loc[399] = scrap_pokemon_moves("Bidoof")
pokemon.loc[400] = scrap_pokemon_moves("Bibarel")
pokemon.loc[401] = scrap_pokemon_moves("Kricketot")
pokemon.loc[402] = scrap_pokemon_moves("Kricketune")

In [29]:
# Shinx to fossils
pokemon.loc[403] = scrap_pokemon_moves("Shinx")
pokemon.loc[404] = scrap_pokemon_moves("Luxio")
pokemon.loc[405] = scrap_pokemon_moves("Luxray")
pokemon.loc[406] = scrap_pokemon_moves("Budew")
pokemon.loc[407] = scrap_pokemon_moves("Roserade")
pokemon.loc[408] = scrap_pokemon_moves("Cranidos")
pokemon.loc[409] = scrap_pokemon_moves("Rampardos")
pokemon.loc[410] = scrap_pokemon_moves("Shieldon")
pokemon.loc[411] = scrap_pokemon_moves("Bastiodon")

In [30]:
# burmy line
pokemon.loc[412] = scrap_pokemon_moves("Burmy")
pokemon.loc[413] = scrap_pokemon_moves("Wormadam")
pokemon.loc[414] = scrap_pokemon_moves("Mothim")
pokemon.loc[415] = scrap_pokemon_moves("Combee")
pokemon.loc[416] = scrap_pokemon_moves("Vespiquen")
pokemon.loc[417] = scrap_pokemon_moves("Pachirisu")
pokemon.loc[418] = scrap_pokemon_moves("Buizel")
pokemon.loc[419] = scrap_pokemon_moves("Floatzel")

In [31]:
# cherubi
pokemon.loc[420] = scrap_pokemon_moves("Cherubi")
pokemon.loc[421] = scrap_pokemon_moves("Cherrim")
pokemon.loc[422] = scrap_pokemon_moves("Shellos")
pokemon.loc[423] = scrap_pokemon_moves("Gastrodon")
pokemon.loc[424] = scrap_pokemon_moves("Ambipom")
pokemon.loc[425] = scrap_pokemon_moves("Drifloon")
pokemon.loc[426] = scrap_pokemon_moves("Drifblim")

In [32]:
# buneary
pokemon.loc[427] = scrap_pokemon_moves("Buneary")
lopunny = scrap_pokemon_moves("Lopunny")
pokemon.loc[428] = lopunny
pokemon.loc[429] = scrap_pokemon_moves("Mismagius")
pokemon.loc[430] = scrap_pokemon_moves("Honchkrow")
pokemon.loc[431] = scrap_pokemon_moves("Glameow")
pokemon.loc[432] = scrap_pokemon_moves("Purugly")
pokemon.loc[433] = scrap_pokemon_moves("Chingling")
pokemon.loc[434] = scrap_pokemon_moves("Stunky")
pokemon.loc[435] = scrap_pokemon_moves("Skuntank")
pokemon.loc[436] = scrap_pokemon_moves("Bronzor")
pokemon.loc[437] = scrap_pokemon_moves("Bronzong")
pokemon.loc[438] = scrap_pokemon_moves("Bonsly")

In [8]:
# MRmime jr
pokemon.loc[439] = scrap_pokemon_moves("Mime_Jr.")
pokemon.loc[440] = scrap_pokemon_moves("Happiny")
pokemon.loc[441] = scrap_pokemon_moves("Chatot")
pokemon.loc[442] = scrap_pokemon_moves("Spiritomb")

In [34]:
# Gible line 
pokemon.loc[443] = scrap_pokemon_moves("Gible")
pokemon.loc[444] = scrap_pokemon_moves("Gabite")
garchomp = scrap_pokemon_moves("Garchomp")
pokemon.loc[445] = garchomp
pokemon.loc[446] = scrap_pokemon_moves("Munchlax")

In [35]:
# riolu line
pokemon.loc[447] = scrap_pokemon_moves("Riolu")
lucario = scrap_pokemon_moves("Lucario")
pokemon.loc[448] = lucario
pokemon.loc[449] = scrap_pokemon_moves("Hippopotas")
pokemon.loc[450] = scrap_pokemon_moves("Hippowdon")
pokemon.loc[451] = scrap_pokemon_moves("Skorupi")
pokemon.loc[452] = scrap_pokemon_moves("Drapion")

In [36]:
# croagunk
pokemon.loc[453] = scrap_pokemon_moves("Croagunk")
pokemon.loc[454] = scrap_pokemon_moves("Toxicroak")
pokemon.loc[455] = scrap_pokemon_moves("Carnivine")
pokemon.loc[456] = scrap_pokemon_moves("Finneon")
pokemon.loc[457] = scrap_pokemon_moves("Lumineon")
pokemon.loc[458] = scrap_pokemon_moves("Mantyke")

In [37]:
# Snover line and new evos
pokemon.loc[459] = scrap_pokemon_moves("Snover")
abomasnow = scrap_pokemon_moves("Abomasnow")
pokemon.loc[460] = abomasnow
pokemon.loc[461] = scrap_pokemon_moves("Weavile")
pokemon.loc[462] = scrap_pokemon_moves("Magnezone")
pokemon.loc[463] = scrap_pokemon_moves("Lickilicky")
pokemon.loc[464] = scrap_pokemon_moves("Rhyperior")
pokemon.loc[465] = scrap_pokemon_moves("Tangrowth")
pokemon.loc[466] = scrap_pokemon_moves("Electivire")
pokemon.loc[467] = scrap_pokemon_moves("Magmortar")
pokemon.loc[468] = scrap_pokemon_moves("Togekiss")
pokemon.loc[469] = scrap_pokemon_moves("Yanmega")
pokemon.loc[470] = scrap_pokemon_moves("Leafeon")
pokemon.loc[471] = scrap_pokemon_moves("Glaceon")
pokemon.loc[472] = scrap_pokemon_moves("Gliscor")
pokemon.loc[473] = scrap_pokemon_moves("Mamoswine")
pokemon.loc[474] = scrap_pokemon_moves("Porygon-Z")

In [38]:
# gallad
gallade = scrap_pokemon_moves("Gallade")
pokemon.loc[475] = gallade
pokemon.loc[476] = scrap_pokemon_moves("Probopass")
pokemon.loc[477] = scrap_pokemon_moves("Dusknoir")
pokemon.loc[478] = scrap_pokemon_moves("Froslass")
pokemon.loc[479] = scrap_pokemon_moves("Rotom")

In [39]:
# legendaries gen 4
pokemon.loc[480] = scrap_pokemon_moves("Uxie")
pokemon.loc[481] = scrap_pokemon_moves("Mesprit")
pokemon.loc[482] = scrap_pokemon_moves("Azelf")
pokemon.loc[483] = scrap_pokemon_moves("Dialga")
pokemon.loc[484] = scrap_pokemon_moves("Palkia")
pokemon.loc[485] = scrap_pokemon_moves("Heatran")
pokemon.loc[486] = scrap_pokemon_moves("Regigigas")
giratina = scrap_pokemon_moves("Giratina")
pokemon.loc[487] = giratina
pokemon.loc[488] = scrap_pokemon_moves("Cresselia")
pokemon.loc[489] = scrap_pokemon_moves("Phione")
pokemon.loc[490] = scrap_pokemon_moves("Manaphy")
pokemon.loc[491] = scrap_pokemon_moves("Darkrai")
shaymin = scrap_pokemon_moves("Shaymin")
pokemon.loc[492] = shaymin
pokemon.loc[493] = scrap_pokemon_moves("Arceus")

In [40]:
# gen 5 victi
pokemon.loc[494] = scrap_pokemon_moves("Victini")

In [41]:
# gen 5 starters snivy
pokemon.loc[495] = scrap_pokemon_moves("Snivy")
pokemon.loc[496] = scrap_pokemon_moves("Servine")
pokemon.loc[497] = scrap_pokemon_moves("Serperior")

In [19]:
# oshawott and tepig
pokemon.loc[498] = scrap_pokemon_moves("Tepig")
pokemon.loc[499] = scrap_pokemon_moves("Pignite")
pokemon.loc[500] = scrap_pokemon_moves("Emboar")
pokemon.loc[501] = scrap_pokemon_moves("Oshawott")
pokemon.loc[502] = scrap_pokemon_moves("Dewott")
pokemon.loc[503] = scrap_pokemon_moves("Samurott")

In [43]:
# patrat
pokemon.loc[504] = scrap_pokemon_moves("Patrat")
pokemon.loc[505] = scrap_pokemon_moves("Watchog")
pokemon.loc[506] = scrap_pokemon_moves("Lillipup")
pokemon.loc[507] = scrap_pokemon_moves("Herdier")
pokemon.loc[508] = scrap_pokemon_moves("Stoutland")
pokemon.loc[509] = scrap_pokemon_moves("Purrloin")
pokemon.loc[510] = scrap_pokemon_moves("Liepard")

In [44]:
#monkeys
pokemon.loc[511] = scrap_pokemon_moves("Pansage")
pokemon.loc[512] = scrap_pokemon_moves("Simisage")
pokemon.loc[513] = scrap_pokemon_moves("Pansear")
pokemon.loc[514] = scrap_pokemon_moves("Simisear")
pokemon.loc[515] = scrap_pokemon_moves("Panpour")
pokemon.loc[516] = scrap_pokemon_moves("Simipour")

In [45]:
# muna and pidove
pokemon.loc[517] = scrap_pokemon_moves("Munna")
pokemon.loc[518] = scrap_pokemon_moves("Musharna")
pokemon.loc[519] = scrap_pokemon_moves("Pidove")
pokemon.loc[520] = scrap_pokemon_moves("Tranquill")
pokemon.loc[521] = scrap_pokemon_moves("Unfezant")
pokemon.loc[522] = scrap_pokemon_moves("Blitzle")
pokemon.loc[523] = scrap_pokemon_moves("Zebstrika")

In [46]:
#roggenrola
pokemon.loc[524] = scrap_pokemon_moves("Roggenrola")
pokemon.loc[525] = scrap_pokemon_moves("Boldore")
pokemon.loc[526] = scrap_pokemon_moves("Gigalith")
pokemon.loc[527] = scrap_pokemon_moves("Woobat")
pokemon.loc[528] = scrap_pokemon_moves("Swoobat")
pokemon.loc[529] = scrap_pokemon_moves("Drilbur")
pokemon.loc[530] = scrap_pokemon_moves("Excadrill")
audino = scrap_pokemon_moves("Audino")
pokemon.loc[531] = audino

In [47]:
# timburr and tympole
pokemon.loc[532] = scrap_pokemon_moves("Timburr")
pokemon.loc[533] = scrap_pokemon_moves("Gurdurr")
pokemon.loc[534] = scrap_pokemon_moves("Conkeldurr")
pokemon.loc[535] = scrap_pokemon_moves("Tympole")
pokemon.loc[536] = scrap_pokemon_moves("Palpitoad")
pokemon.loc[537] = scrap_pokemon_moves("Seismitoad")
pokemon.loc[538] = scrap_pokemon_moves("Throh")
pokemon.loc[539] = scrap_pokemon_moves("Sawk")

In [48]:
# swaddle
pokemon.loc[540] = scrap_pokemon_moves("Sewaddle")
pokemon.loc[541] = scrap_pokemon_moves("Swadloon")
pokemon.loc[542] = scrap_pokemon_moves("Leavanny")
venipede = scrap_pokemon_moves("Venipede")
pokemon.loc[543] = venipede
whirlipede = scrap_pokemon_moves("Whirlipede")
pokemon.loc[544] = whirlipede
scolipede = scrap_pokemon_moves("Scolipede")
pokemon.loc[545] = scolipede

In [49]:
# cottonee
pokemon.loc[546] = scrap_pokemon_moves("Cottonee")
pokemon.loc[547] = scrap_pokemon_moves("Whimsicott")
pokemon.loc[548] = scrap_pokemon_moves("Petilil")

In [50]:
# lilligant
lilligant = scrap_pokemon_moves("Lilligant")
pokemon.loc[549] = lilligant
basculin = scrap_pokemon_moves("Basculin")
pokemon.loc[550] = basculin

In [51]:
# sandile
pokemon.loc[551] = scrap_pokemon_moves("Sandile")
pokemon.loc[552] = scrap_pokemon_moves("Krokorok")
pokemon.loc[553] = scrap_pokemon_moves("Krookodile")
pokemon.loc[554] = scrap_pokemon_moves("Darumaka")
darmanitan = scrap_pokemon_moves("Darmanitan")
pokemon.loc[555] = darmanitan

In [52]:
# maractus 
pokemon.loc[556] = scrap_pokemon_moves("Maractus")
pokemon.loc[557] = scrap_pokemon_moves("Dwebble")
pokemon.loc[558] = scrap_pokemon_moves("Crustle")
pokemon.loc[559] = scrap_pokemon_moves("Scraggy")
pokemon.loc[560] = scrap_pokemon_moves("Scrafty")
pokemon.loc[561] = scrap_pokemon_moves("Sigilyph")

In [53]:
# yamask
pokemon.loc[562] = scrap_pokemon_moves("Yamask")
pokemon.loc[563] = scrap_pokemon_moves("Cofagrigus")
pokemon.loc[564] = scrap_pokemon_moves("Tirtouga")
pokemon.loc[565] = scrap_pokemon_moves("Carracosta")
pokemon.loc[566] = scrap_pokemon_moves("Archen")
pokemon.loc[567] = scrap_pokemon_moves("Archeops")
pokemon.loc[568] = scrap_pokemon_moves("Trubbish")
pokemon.loc[569] = scrap_pokemon_moves("Garbodor")

In [54]:
# zorua
pokemon.loc[570] = scrap_pokemon_moves("Zorua")
pokemon.loc[571] = scrap_pokemon_moves("Zoroark")

In [55]:
# minccinp 
pokemon.loc[572] = scrap_pokemon_moves("Minccino")
pokemon.loc[573] = scrap_pokemon_moves("Cinccino")
pokemon.loc[574] = scrap_pokemon_moves("Gothita")
pokemon.loc[575] = scrap_pokemon_moves("Gothorita")
pokemon.loc[576] = scrap_pokemon_moves("Gothitelle")
pokemon.loc[577] = scrap_pokemon_moves("Solosis")
pokemon.loc[578] = scrap_pokemon_moves("Duosion")
pokemon.loc[579] = scrap_pokemon_moves("Reuniclus")

In [56]:
#ducklett
pokemon.loc[580] = scrap_pokemon_moves("Ducklett")
pokemon.loc[581] = scrap_pokemon_moves("Swanna")
pokemon.loc[582] = scrap_pokemon_moves("Vanillite")
pokemon.loc[583] = scrap_pokemon_moves("Vanillish")
pokemon.loc[584] = scrap_pokemon_moves("Vanilluxe")
pokemon.loc[585] = scrap_pokemon_moves("Deerling")
pokemon.loc[586] = scrap_pokemon_moves("Sawsbuck")

In [57]:
# emolga
pokemon.loc[587] = scrap_pokemon_moves("Emolga")
pokemon.loc[588] = scrap_pokemon_moves("Karrablast")
pokemon.loc[589] = scrap_pokemon_moves("Escavalier")
pokemon.loc[590] = scrap_pokemon_moves("Foongus")
pokemon.loc[591] = scrap_pokemon_moves("Amoonguss")
pokemon.loc[592] = scrap_pokemon_moves("Frillish")
pokemon.loc[593] = scrap_pokemon_moves("Jellicent")
pokemon.loc[594] = scrap_pokemon_moves("Alomomola")
pokemon.loc[595] = scrap_pokemon_moves("Joltik")
pokemon.loc[596] = scrap_pokemon_moves("Galvantula")
pokemon.loc[597] = scrap_pokemon_moves("Ferroseed")
pokemon.loc[598] = scrap_pokemon_moves("Ferrothorn")
pokemon.loc[599] = scrap_pokemon_moves("Klink")
pokemon.loc[600] = scrap_pokemon_moves("Klang")
pokemon.loc[601] = scrap_pokemon_moves("Klinklang")

In [58]:
# tynamo
pokemon.loc[602] = scrap_pokemon_moves("Tynamo")
pokemon.loc[603] = scrap_pokemon_moves("Eelektrik")
pokemon.loc[604] = scrap_pokemon_moves("Eelektross")
pokemon.loc[605] = scrap_pokemon_moves("Elgyem")
pokemon.loc[606] = scrap_pokemon_moves("Beheeyem")
pokemon.loc[607] = scrap_pokemon_moves("Litwick")
pokemon.loc[608] = scrap_pokemon_moves("Lampent")
pokemon.loc[609] = scrap_pokemon_moves("Chandelure")

In [59]:
# axew
pokemon.loc[610] = scrap_pokemon_moves("Axew")
pokemon.loc[611] = scrap_pokemon_moves("Fraxure")
pokemon.loc[612] = scrap_pokemon_moves("Haxorus")
pokemon.loc[613] = scrap_pokemon_moves("Cubchoo")
pokemon.loc[614] = scrap_pokemon_moves("Beartic")
pokemon.loc[615] = scrap_pokemon_moves("Cryogonal")
pokemon.loc[616] = scrap_pokemon_moves("Shelmet")
pokemon.loc[617] = scrap_pokemon_moves("Accelgor")

In [60]:
# stunkfish 
pokemon.loc[618] = scrap_pokemon_moves("Stunfisk")

In [61]:
# meinfoo
pokemon.loc[619] = scrap_pokemon_moves("Mienfoo")
pokemon.loc[620] = scrap_pokemon_moves("Mienshao")
pokemon.loc[621] = scrap_pokemon_moves("Druddigon")
pokemon.loc[622] = scrap_pokemon_moves("Golett")
pokemon.loc[623] = scrap_pokemon_moves("Golurk")
pokemon.loc[624] = scrap_pokemon_moves("Pawniard")
pokemon.loc[625] = scrap_pokemon_moves("Bisharp")
pokemon.loc[626] = scrap_pokemon_moves("Bouffalant")
pokemon.loc[627] = scrap_pokemon_moves("Rufflet")
pokemon.loc[628] = scrap_pokemon_moves("Braviary")

In [62]:
# vullaby
pokemon.loc[629] = scrap_pokemon_moves("Vullaby")
pokemon.loc[630] = scrap_pokemon_moves("Mandibuzz")
pokemon.loc[631] = scrap_pokemon_moves("Heatmor")
pokemon.loc[632] = scrap_pokemon_moves("Durant")
pokemon.loc[633] = scrap_pokemon_moves("Deino")
pokemon.loc[634] = scrap_pokemon_moves("Zweilous")
pokemon.loc[635] = scrap_pokemon_moves("Hydreigon")
pokemon.loc[636] = scrap_pokemon_moves("Larvesta")
pokemon.loc[637] = scrap_pokemon_moves("Volcarona")

In [63]:
# legendaries
pokemon.loc[638] = scrap_pokemon_moves("Cobalion")
pokemon.loc[639] = scrap_pokemon_moves("Terrakion")
pokemon.loc[640] = scrap_pokemon_moves("Virizion")
pokemon.loc[641] = scrap_pokemon_moves("Tornadus")
pokemon.loc[642] = scrap_pokemon_moves("Thundurus")
pokemon.loc[643] = scrap_pokemon_moves("Reshiram")
pokemon.loc[644] = scrap_pokemon_moves("Zekrom")
pokemon.loc[645] = scrap_pokemon_moves("Landorus")
pokemon.loc[646] = scrap_pokemon_moves("Kyurem")
pokemon.loc[647] = scrap_pokemon_moves("Keldeo")
pokemon.loc[648] = scrap_pokemon_moves("Meloetta")
pokemon.loc[649] = scrap_pokemon_moves("Genesect")

In [64]:
# gen 6 starters 
pokemon.loc[650] = scrap_pokemon_moves("Chespin")
pokemon.loc[651] = scrap_pokemon_moves("Quilladin")
pokemon.loc[652] = scrap_pokemon_moves("Chesnaught")
pokemon.loc[653] = scrap_pokemon_moves("Fennekin")
pokemon.loc[654] = scrap_pokemon_moves("Braixen")
pokemon.loc[655] = scrap_pokemon_moves("Delphox")
pokemon.loc[656] = scrap_pokemon_moves("Froakie")
pokemon.loc[657] = scrap_pokemon_moves("Frogadier")
pokemon.loc[658] = scrap_pokemon_moves("Greninja")

In [65]:
# bunnelby
pokemon.loc[659] = scrap_pokemon_moves("Bunnelby")
pokemon.loc[660] = scrap_pokemon_moves("Diggersby")
pokemon.loc[661] = scrap_pokemon_moves("Fletchling")
pokemon.loc[662] = scrap_pokemon_moves("Fletchinder")
pokemon.loc[663] = scrap_pokemon_moves("Talonflame")

In [66]:
# scatterbug
pokemon.loc[664] = scrap_pokemon_moves("Scatterbug")
pokemon.loc[665] = scrap_pokemon_moves("Spewpa")
pokemon.loc[666] = scrap_pokemon_moves("Vivillon")
pokemon.loc[667] = scrap_pokemon_moves("Litleo")
pokemon.loc[668] = scrap_pokemon_moves("Pyroar")
pokemon.loc[669] = scrap_pokemon_moves("Flabébé")
pokemon.loc[670] = scrap_pokemon_moves("Floette")
pokemon.loc[671] = scrap_pokemon_moves("Florges")

In [67]:
# skiddo
pokemon.loc[672] = scrap_pokemon_moves("Skiddo")
pokemon.loc[673] = scrap_pokemon_moves("Gogoat")
pokemon.loc[674] = scrap_pokemon_moves("Pancham")
pokemon.loc[675] = scrap_pokemon_moves("Pangoro")
pokemon.loc[676] = scrap_pokemon_moves("Furfrou")
pokemon.loc[677] = scrap_pokemon_moves("Espurr")
pokemon.loc[678] = scrap_pokemon_moves("Meowstic")
pokemon.loc[679] = scrap_pokemon_moves("Honedge")
pokemon.loc[680] = scrap_pokemon_moves("Doublade")
pokemon.loc[681] = scrap_pokemon_moves("Aegislash")

In [68]:
# spritzee
pokemon.loc[682] = scrap_pokemon_moves("Spritzee")
pokemon.loc[683] = scrap_pokemon_moves("Aromatisse")
pokemon.loc[684] = scrap_pokemon_moves("Swirlix")
pokemon.loc[685] = scrap_pokemon_moves("Slurpuff")
pokemon.loc[686] = scrap_pokemon_moves("Inkay")
pokemon.loc[687] = scrap_pokemon_moves("Malamar")
pokemon.loc[688] = scrap_pokemon_moves("Binacle")
pokemon.loc[689] = scrap_pokemon_moves("Barbaracle")
pokemon.loc[690] = scrap_pokemon_moves("Skrelp")
pokemon.loc[691] = scrap_pokemon_moves("Dragalge")
pokemon.loc[692] = scrap_pokemon_moves("Clauncher")
pokemon.loc[693] = scrap_pokemon_moves("Clawitzer")
pokemon.loc[694] = scrap_pokemon_moves("Helioptile")
pokemon.loc[695] = scrap_pokemon_moves("Heliolisk")
pokemon.loc[696] = scrap_pokemon_moves("Tyrunt")
pokemon.loc[697] = scrap_pokemon_moves("Tyrantrum")
pokemon.loc[698] = scrap_pokemon_moves("Amaura")
pokemon.loc[699] = scrap_pokemon_moves("Aurorus")
pokemon.loc[700] = scrap_pokemon_moves("Sylveon")

In [69]:
# hawlucha
pokemon.loc[701] = scrap_pokemon_moves("Hawlucha")
pokemon.loc[702] = scrap_pokemon_moves("Dedenne")
pokemon.loc[703] = scrap_pokemon_moves("Carbink")
pokemon.loc[704] = scrap_pokemon_moves("Goomy")
pokemon.loc[705] = scrap_pokemon_moves("Sliggoo")
pokemon.loc[706] = scrap_pokemon_moves("Goodra")

In [70]:
#klefki
pokemon.loc[707] = scrap_pokemon_moves("Klefki")
pokemon.loc[708] = scrap_pokemon_moves("Phantump")
pokemon.loc[709] = scrap_pokemon_moves("Trevenant")
pokemon.loc[710] = scrap_pokemon_moves("Pumpkaboo")
pokemon.loc[711] = scrap_pokemon_moves("Gourgeist")
pokemon.loc[712] = scrap_pokemon_moves("Bergmite")
pokemon.loc[713] = scrap_pokemon_moves("Avalugg")

In [72]:
# Noibat
pokemon.loc[714] = scrap_pokemon_moves("Noibat")
pokemon.loc[715] = scrap_pokemon_moves("Noivern")
pokemon.loc[716] = scrap_pokemon_moves("Xerneas")
pokemon.loc[717] = scrap_pokemon_moves("Yveltal")
pokemon.loc[718] = scrap_pokemon_moves("Zygarde")
pokemon.loc[719] = scrap_pokemon_moves("Diancie")
pokemon.loc[720] = scrap_pokemon_moves("Hoopa")
pokemon.loc[721] = scrap_pokemon_moves("Volcanion")

In [73]:
# gen 7 starters 
pokemon.loc[722] = scrap_pokemon_moves("Rowlet")
pokemon.loc[723] = scrap_pokemon_moves("Dartrix")
pokemon.loc[724] = scrap_pokemon_moves("Decidueye")
pokemon.loc[725] = scrap_pokemon_moves("Litten")
pokemon.loc[726] = scrap_pokemon_moves("Torracat")
pokemon.loc[727] = scrap_pokemon_moves("Incineroar")
pokemon.loc[728] = scrap_pokemon_moves("Popplio")
pokemon.loc[729] = scrap_pokemon_moves("Brionne")

In [74]:
# pikipek line 
pokemon.loc[730] = scrap_pokemon_moves("Primarina")
pokemon.loc[731] = scrap_pokemon_moves("Pikipek")
pokemon.loc[732] = scrap_pokemon_moves("Trumbeak")
pokemon.loc[733] = scrap_pokemon_moves("Toucannon")
pokemon.loc[734] = scrap_pokemon_moves("Yungoos")
pokemon.loc[735] = scrap_pokemon_moves("Gumshoos")


In [75]:
# grubbin line
pokemon.loc[736] = scrap_pokemon_moves("Grubbin")
pokemon.loc[737] = scrap_pokemon_moves("Charjabug")
pokemon.loc[738] = scrap_pokemon_moves("Vikavolt")
pokemon.loc[739] = scrap_pokemon_moves("Crabrawler")
pokemon.loc[740] = scrap_pokemon_moves("Crabominable")
pokemon.loc[741] = scrap_pokemon_moves("Oricorio")
pokemon.loc[742] = scrap_pokemon_moves("Cutiefly")
pokemon.loc[743] = scrap_pokemon_moves("Ribombee")

In [76]:
# rockruff
pokemon.loc[744] = scrap_pokemon_moves("Rockruff")
pokemon.loc[745] = scrap_pokemon_moves("Lycanroc")
pokemon.loc[746] = scrap_pokemon_moves("Wishiwashi")
pokemon.loc[747] = scrap_pokemon_moves("Mareanie")
pokemon.loc[748] = scrap_pokemon_moves("Toxapex")

In [77]:
# Mudbray
pokemon.loc[749] = scrap_pokemon_moves("Mudbray")
pokemon.loc[750] = scrap_pokemon_moves("Mudsdale")
pokemon.loc[751] = scrap_pokemon_moves("Dewpider")
pokemon.loc[752] = scrap_pokemon_moves("Araquanid")
pokemon.loc[753] = scrap_pokemon_moves("Fomantis")
pokemon.loc[754] = scrap_pokemon_moves("Lurantis")
pokemon.loc[755] = scrap_pokemon_moves("Morelull")
pokemon.loc[756] = scrap_pokemon_moves("Shiinotic")
pokemon.loc[757] = scrap_pokemon_moves("Salandit")
pokemon.loc[758] = scrap_pokemon_moves("Salazzle")

In [78]:
# stufful
pokemon.loc[759] = scrap_pokemon_moves("Stufful")
pokemon.loc[760] = scrap_pokemon_moves("Bewear")
pokemon.loc[761] = scrap_pokemon_moves("Bounsweet")
pokemon.loc[762] = scrap_pokemon_moves("Steenee")
pokemon.loc[763] = scrap_pokemon_moves("Tsareena")

In [79]:
# comfey 
pokemon.loc[764] = scrap_pokemon_moves("Comfey")
pokemon.loc[765] = scrap_pokemon_moves("Oranguru")
pokemon.loc[766] = scrap_pokemon_moves("Passimian")
pokemon.loc[767] = scrap_pokemon_moves("Wimpod")
pokemon.loc[768] = scrap_pokemon_moves("Golisopod")
pokemon.loc[769] = scrap_pokemon_moves("Sandygast")
pokemon.loc[770] = scrap_pokemon_moves("Palossand")

In [9]:
# pyukumuku
pokemon.loc[771] = scrap_pokemon_moves("Pyukumuku")
pokemon.loc[772] = scrap_pokemon_moves("Type:_Null")
pokemon.loc[773] = scrap_pokemon_moves("Silvally")
pokemon.loc[774] = scrap_pokemon_moves("Minior")
pokemon.loc[775] = scrap_pokemon_moves("Komala")
pokemon.loc[776] = scrap_pokemon_moves("Turtonator")
pokemon.loc[777] = scrap_pokemon_moves("Togedemaru")
pokemon.loc[778] = scrap_pokemon_moves("Mimikyu")

In [81]:
# bruxish 
pokemon.loc[779] = scrap_pokemon_moves("Bruxish")
pokemon.loc[780] = scrap_pokemon_moves("Drampa")
pokemon.loc[781] = scrap_pokemon_moves("Dhelmise")
pokemon.loc[782] = scrap_pokemon_moves("Jangmo-o")
pokemon.loc[783] = scrap_pokemon_moves("Hakamo-o")
pokemon.loc[784] = scrap_pokemon_moves("Kommo-o")

In [10]:
# legendaries
pokemon.loc[785] = scrap_pokemon_moves("Tapu_Koko")
pokemon.loc[786] = scrap_pokemon_moves("Tapu_Lele")
pokemon.loc[787] = scrap_pokemon_moves("Tapu_Bulu")
pokemon.loc[788] = scrap_pokemon_moves("Tapu_Fini")
pokemon.loc[789] = scrap_pokemon_moves("Cosmog")
pokemon.loc[790] = scrap_pokemon_moves("Cosmoem")
pokemon.loc[791] = scrap_pokemon_moves("Solgaleo")
pokemon.loc[792] = scrap_pokemon_moves("Lunala")

In [83]:
# ultra beasts
pokemon.loc[793] = scrap_pokemon_moves("Nihilego")
pokemon.loc[794] = scrap_pokemon_moves("Buzzwole")
pokemon.loc[795] = scrap_pokemon_moves("Pheromosa")
pokemon.loc[796] = scrap_pokemon_moves("Xurkitree")
pokemon.loc[797] = scrap_pokemon_moves("Celesteela")
pokemon.loc[798] = scrap_pokemon_moves("Kartana")
pokemon.loc[799] = scrap_pokemon_moves("Guzzlord")
pokemon.loc[800] = scrap_pokemon_moves("Necrozma")
pokemon.loc[801] = scrap_pokemon_moves("Magearna")
pokemon.loc[802] = scrap_pokemon_moves("Marshadow")
pokemon.loc[803] = scrap_pokemon_moves("Poipole")
pokemon.loc[804] = scrap_pokemon_moves("Naganadel")
pokemon.loc[805] = scrap_pokemon_moves("Stakataka")
pokemon.loc[806] = scrap_pokemon_moves("Blacephalon")
pokemon.loc[807] = scrap_pokemon_moves("Zeraora")
pokemon.loc[808] = scrap_pokemon_moves("Meltan")
pokemon.loc[809] = scrap_pokemon_moves("Melmetal")

In [84]:
# gen 8 starters 
pokemon.loc[810] = scrap_pokemon_moves("Grookey")
pokemon.loc[811] = scrap_pokemon_moves("Thwackey")
pokemon.loc[812] = scrap_pokemon_moves("Rillaboom")
pokemon.loc[813] = scrap_pokemon_moves("Scorbunny")
pokemon.loc[814] = scrap_pokemon_moves("Raboot")
pokemon.loc[815] = scrap_pokemon_moves("Cinderace")
pokemon.loc[816] = scrap_pokemon_moves("Sobble")
pokemon.loc[817] = scrap_pokemon_moves("Drizzile")
pokemon.loc[818] = scrap_pokemon_moves("Inteleon")

In [85]:
# skwovet
pokemon.loc[819] = scrap_pokemon_moves("Skwovet")
pokemon.loc[820] = scrap_pokemon_moves("Greedent")
pokemon.loc[821] = scrap_pokemon_moves("Rookidee")
pokemon.loc[822] = scrap_pokemon_moves("Corvisquire")
pokemon.loc[823] = scrap_pokemon_moves("Corviknight")
pokemon.loc[824] = scrap_pokemon_moves("Blipbug")
pokemon.loc[825] = scrap_pokemon_moves("Dottler")
pokemon.loc[826] = scrap_pokemon_moves("Orbeetle")
pokemon.loc[827] = scrap_pokemon_moves("Nickit")
pokemon.loc[828] = scrap_pokemon_moves("Thievul")

In [86]:
#gossifleur
pokemon.loc[829] = scrap_pokemon_moves("Gossifleur")
pokemon.loc[830] = scrap_pokemon_moves("Eldegoss")
pokemon.loc[831] = scrap_pokemon_moves("Wooloo")
pokemon.loc[832] = scrap_pokemon_moves("Dubwool")
pokemon.loc[833] = scrap_pokemon_moves("Chewtle")
pokemon.loc[834] = scrap_pokemon_moves("Drednaw")
pokemon.loc[835] = scrap_pokemon_moves("Yamper")
pokemon.loc[836] = scrap_pokemon_moves("Boltund")
pokemon.loc[837] = scrap_pokemon_moves("Rolycoly")
pokemon.loc[838] = scrap_pokemon_moves("Carkol")
pokemon.loc[839] = scrap_pokemon_moves("Coalossal")

In [87]:
# applin'
pokemon.loc[840] = scrap_pokemon_moves("Applin")
pokemon.loc[841] = scrap_pokemon_moves("Flapple")
pokemon.loc[842] = scrap_pokemon_moves("Appletun")
pokemon.loc[843] = scrap_pokemon_moves("Silicobra")
pokemon.loc[844] = scrap_pokemon_moves("Sandaconda")
pokemon.loc[845] = scrap_pokemon_moves("Cramorant")
pokemon.loc[846] = scrap_pokemon_moves("Arrokuda")
pokemon.loc[847] = scrap_pokemon_moves("Barraskewda")
pokemon.loc[848] = scrap_pokemon_moves("Toxel")
pokemon.loc[849] = scrap_pokemon_moves("Toxtricity")

In [88]:
#sizzlipede
pokemon.loc[850] = scrap_pokemon_moves("Sizzlipede")
pokemon.loc[851] = scrap_pokemon_moves("Centiskorch")
pokemon.loc[852] = scrap_pokemon_moves("Clobbopus")
pokemon.loc[853] = scrap_pokemon_moves("Grapploct")
pokemon.loc[854] = scrap_pokemon_moves("Sinistea")
pokemon.loc[855] = scrap_pokemon_moves("Polteageist")
pokemon.loc[856] = scrap_pokemon_moves("Hatenna")
pokemon.loc[857] = scrap_pokemon_moves("Hattrem")
pokemon.loc[858] = scrap_pokemon_moves("Hatterene")

In [11]:
# impidimp
pokemon.loc[859] = scrap_pokemon_moves("Impidimp")
pokemon.loc[860] = scrap_pokemon_moves("Morgrem")
pokemon.loc[861] = scrap_pokemon_moves("Grimmsnarl")
pokemon.loc[862] = scrap_pokemon_moves("Obstagoon")
pokemon.loc[863] = scrap_pokemon_moves("Perrserker")
pokemon.loc[864] = scrap_pokemon_moves("Cursola")
pokemon.loc[865] = scrap_pokemon_moves("Sirfetch'd")
pokemon.loc[866] = scrap_pokemon_moves("Mr._Rime")
pokemon.loc[867] = scrap_pokemon_moves("Runerigus")
pokemon.loc[868] = scrap_pokemon_moves("Milcery")
pokemon.loc[869] = scrap_pokemon_moves("Alcremie")
pokemon.loc[870] = scrap_pokemon_moves("Falinks")
pokemon.loc[871] = scrap_pokemon_moves("Pincurchin")

In [90]:
# snom
pokemon.loc[872] = scrap_pokemon_moves("Snom")
pokemon.loc[873] = scrap_pokemon_moves("Frosmoth")
pokemon.loc[874] = scrap_pokemon_moves("Stonjourner")
pokemon.loc[875] = scrap_pokemon_moves("Eiscue")
pokemon.loc[876] = scrap_pokemon_moves("Indeedee")
pokemon.loc[877] = scrap_pokemon_moves("Morpeko")
pokemon.loc[878] = scrap_pokemon_moves("Cufant")
pokemon.loc[879] = scrap_pokemon_moves("Copperajah")

In [91]:
# fossils
pokemon.loc[880] = scrap_pokemon_moves("Dracozolt")
pokemon.loc[881] = scrap_pokemon_moves("Arctozolt")
pokemon.loc[882] = scrap_pokemon_moves("Dracovish")
pokemon.loc[883] = scrap_pokemon_moves("Arctovish")
pokemon.loc[884] = scrap_pokemon_moves("Duraludon")
pokemon.loc[885] = scrap_pokemon_moves("Dreepy")
pokemon.loc[886] = scrap_pokemon_moves("Drakloak")
pokemon.loc[887] = scrap_pokemon_moves("Dragapult")


In [92]:
#legendaries
pokemon.loc[888] = scrap_pokemon_moves("Zacian")
pokemon.loc[889] = scrap_pokemon_moves("Zamazenta")
pokemon.loc[890] = scrap_pokemon_moves("Eternatus")
pokemon.loc[891] = scrap_pokemon_moves("Kubfu")
pokemon.loc[892] = scrap_pokemon_moves("Urshifu")
pokemon.loc[893] = scrap_pokemon_moves("Zarude")
pokemon.loc[894] = scrap_pokemon_moves("Regieleki")
pokemon.loc[895] = scrap_pokemon_moves("Regidrago")
pokemon.loc[896] = scrap_pokemon_moves("Glastrier")
pokemon.loc[897] = scrap_pokemon_moves("Spectrier")
pokemon.loc[898] = scrap_pokemon_moves("Calyrex")

In [93]:
# hisuin 
pokemon.loc[899] = scrap_pokemon_moves("Wyrdeer")
pokemon.loc[900] = scrap_pokemon_moves("Kleavor")
pokemon.loc[901] = scrap_pokemon_moves("Ursaluna")
pokemon.loc[902] = scrap_pokemon_moves("Basculegion")
pokemon.loc[903] = scrap_pokemon_moves("Sneasler")
pokemon.loc[904] = scrap_pokemon_moves("Overqwil")
pokemon.loc[905] = scrap_pokemon_moves("Enamorus")

In [94]:
# gen 9 starters
pokemon.loc[906] = scrap_pokemon_moves("Sprigatito")
pokemon.loc[907] = scrap_pokemon_moves("Floragato")
pokemon.loc[908] = scrap_pokemon_moves("Meowscarada")
pokemon.loc[909] = scrap_pokemon_moves("Fuecoco")
pokemon.loc[910] = scrap_pokemon_moves("Crocalor")
pokemon.loc[911] = scrap_pokemon_moves("Skeledirge")
pokemon.loc[912] = scrap_pokemon_moves("Quaxly")
pokemon.loc[913] = scrap_pokemon_moves("Quaxwell")
pokemon.loc[914] = scrap_pokemon_moves("Quaquaval")

In [95]:
#lechonk
pokemon.loc[915] = scrap_pokemon_moves("Lechonk")
pokemon.loc[916] = scrap_pokemon_moves("Oinkologne")
pokemon.loc[917] = scrap_pokemon_moves("Tarountula")
pokemon.loc[918] = scrap_pokemon_moves("Spidops")
pokemon.loc[919] = scrap_pokemon_moves("Nymble")
pokemon.loc[920] = scrap_pokemon_moves("Lokix")
pokemon.loc[921] = scrap_pokemon_moves("Pawmi")
pokemon.loc[922] = scrap_pokemon_moves("Pawmo")
pokemon.loc[923] = scrap_pokemon_moves("Pawmot")
pokemon.loc[924] = scrap_pokemon_moves("Tandemaus")
pokemon.loc[925] = scrap_pokemon_moves("Maushold")
pokemon.loc[926] = scrap_pokemon_moves("Fidough")
pokemon.loc[927] = scrap_pokemon_moves("Dachsbun")
pokemon.loc[928] = scrap_pokemon_moves("Smoliv")
pokemon.loc[929] = scrap_pokemon_moves("Dolliv")
pokemon.loc[930] = scrap_pokemon_moves("Arboliva")

In [96]:
# nacli
pokemon.loc[931] = scrap_pokemon_moves("Squawkabilly")
pokemon.loc[932] = scrap_pokemon_moves("Nacli")
pokemon.loc[933] = scrap_pokemon_moves("Naclstack")
pokemon.loc[934] = scrap_pokemon_moves("Garganacl")

In [97]:
# charcadet
pokemon.loc[935] = scrap_pokemon_moves("Charcadet")
pokemon.loc[936] = scrap_pokemon_moves("Armarouge")
pokemon.loc[937] = scrap_pokemon_moves("Ceruledge")
pokemon.loc[938] = scrap_pokemon_moves("Tadbulb")
pokemon.loc[939] = scrap_pokemon_moves("Bellibolt")
pokemon.loc[940] = scrap_pokemon_moves("Wattrel")
pokemon.loc[941] = scrap_pokemon_moves("Kilowattrel")
pokemon.loc[942] = scrap_pokemon_moves("Maschiff")
pokemon.loc[943] = scrap_pokemon_moves("Mabosstiff")
pokemon.loc[944] = scrap_pokemon_moves("Shroodle")
pokemon.loc[945] = scrap_pokemon_moves("Grafaiai")
pokemon.loc[946] = scrap_pokemon_moves("Bramblin")
pokemon.loc[947] = scrap_pokemon_moves("Brambleghast")
pokemon.loc[948] = scrap_pokemon_moves("Toedscool")
pokemon.loc[949] = scrap_pokemon_moves("Toedscruel")
pokemon.loc[950] = scrap_pokemon_moves("Klawf")

In [98]:
# capskid
pokemon.loc[951] = scrap_pokemon_moves("Capsakid")
pokemon.loc[952] = scrap_pokemon_moves("Scovillain")
pokemon.loc[953] = scrap_pokemon_moves("Rellor")
pokemon.loc[954] = scrap_pokemon_moves("Rabsca")
pokemon.loc[955] = scrap_pokemon_moves("Flittle")
pokemon.loc[956] = scrap_pokemon_moves("Espathra")
pokemon.loc[957] = scrap_pokemon_moves("Tinkatink")
pokemon.loc[958] = scrap_pokemon_moves("Tinkatuff")
pokemon.loc[959] = scrap_pokemon_moves("Tinkaton")
pokemon.loc[960] = scrap_pokemon_moves("Wiglett")
pokemon.loc[961] = scrap_pokemon_moves("Wugtrio")
pokemon.loc[962] = scrap_pokemon_moves("Bombirdier")

In [99]:
# finizen
pokemon.loc[963] = scrap_pokemon_moves("Finizen")
pokemon.loc[964] = scrap_pokemon_moves("Palafin")
pokemon.loc[965] = scrap_pokemon_moves("Varoom")
pokemon.loc[966] = scrap_pokemon_moves("Revavroom")
pokemon.loc[967] = scrap_pokemon_moves("Cyclizar")
pokemon.loc[968] = scrap_pokemon_moves("Orthworm")
pokemon.loc[969] = scrap_pokemon_moves("Glimmet")
pokemon.loc[970] = scrap_pokemon_moves("Glimmora")
pokemon.loc[971] = scrap_pokemon_moves("Greavard")
pokemon.loc[972] = scrap_pokemon_moves("Houndstone")
pokemon.loc[973] = scrap_pokemon_moves("Flamigo")
pokemon.loc[974] = scrap_pokemon_moves("Cetoddle")
pokemon.loc[975] = scrap_pokemon_moves("Cetitan")
pokemon.loc[976] = scrap_pokemon_moves("Veluza")
pokemon.loc[977] = scrap_pokemon_moves("Dondozo")
pokemon.loc[978] = scrap_pokemon_moves("Tatsugiri")

In [100]:
# annihalape
pokemon.loc[979] = scrap_pokemon_moves("Annihilape")
pokemon.loc[980] = scrap_pokemon_moves("Clodsire")
pokemon.loc[981] = scrap_pokemon_moves("Farigiraf")
pokemon.loc[982] = scrap_pokemon_moves("Dudunsparce")
pokemon.loc[983] = scrap_pokemon_moves("Kingambit")

In [20]:
# 
pokemon.loc[984] = scrap_pokemon_moves("Great_Tusk")
pokemon.loc[985] = scrap_pokemon_moves("Scream_Tail")
pokemon.loc[986] = scrap_pokemon_moves("Brute_Bonnet")
pokemon.loc[987] = scrap_pokemon_moves("Flutter_Mane")

In [12]:
pokemon.loc[988] = scrap_pokemon_moves("Slither_Wing")
pokemon.loc[989] = scrap_pokemon_moves("Sandy_Shocks")
pokemon.loc[990] = scrap_pokemon_moves("Iron_Treads")
pokemon.loc[991] = scrap_pokemon_moves("Iron_Bundle")
pokemon.loc[992] = scrap_pokemon_moves("Iron_Hands")
pokemon.loc[993] = scrap_pokemon_moves("Iron_Jugulis")
pokemon.loc[994] = scrap_pokemon_moves("Iron_Moth")
pokemon.loc[995] = scrap_pokemon_moves("Iron_Thorns")
pokemon.loc[996] = scrap_pokemon_moves("Frigibax")
pokemon.loc[997] = scrap_pokemon_moves("Arctibax")
pokemon.loc[998] = scrap_pokemon_moves("Baxcalibur")

In [103]:
pokemon.loc[999] = scrap_pokemon_moves("Gimmighoul")
pokemon.loc[1000] = scrap_pokemon_moves("Gholdengo")

In [13]:
pokemon.loc[1001] = scrap_pokemon_moves("Wo-Chien")
pokemon.loc[1002] = scrap_pokemon_moves("Chien-Pao")
pokemon.loc[1003] = scrap_pokemon_moves("Ting-Lu")
pokemon.loc[1004] = scrap_pokemon_moves("Chi-Yu")
pokemon.loc[1005] = scrap_pokemon_moves("Roaring_Moon")

In [14]:
pokemon.loc[1006] = scrap_pokemon_moves("Iron_Valiant")
pokemon.loc[1007] = scrap_pokemon_moves("Koraidon")
pokemon.loc[1008] = scrap_pokemon_moves("Miraidon")
pokemon.loc[1009] = scrap_pokemon_moves("Walking_Wake")
pokemon.loc[1010] = scrap_pokemon_moves("Iron_Leaves")

In [106]:
pokemon.loc[1011] = scrap_pokemon_moves("Dipplin")
pokemon.loc[1012] = scrap_pokemon_moves("Poltchageist")
pokemon.loc[1013] = scrap_pokemon_moves("Sinistcha")
pokemon.loc[1014] = scrap_pokemon_moves("Okidogi")
pokemon.loc[1015] = scrap_pokemon_moves("Munkidori")
pokemon.loc[1016] = scrap_pokemon_moves("Fezandipiti")
pokemon.loc[1017] = scrap_pokemon_moves("Ogerpon")

In [15]:
pokemon.loc[1018] = scrap_pokemon_moves("Archaludon")
pokemon.loc[1019] = scrap_pokemon_moves("Hydrapple")
pokemon.loc[1020] = scrap_pokemon_moves("Gouging_Fire")
pokemon.loc[1021] = scrap_pokemon_moves("Raging_Bolt")
pokemon.loc[1022] = scrap_pokemon_moves("Iron_Boulder")
pokemon.loc[1023] = scrap_pokemon_moves("Iron_Crown")
pokemon.loc[1024] = scrap_pokemon_moves("Terapagos")
pokemon.loc[1025] = scrap_pokemon_moves("Pecharunt")

In [15]:
pokemon.to_csv("../pokemon_moves.csv")
pokemon

,name,moves
1,Bulbasaur,"[Tackle, Growl, Vine Whip, Growth, Leech Seed,..."
2,Ivysaur,"[Tackle, Growl, Vine Whip, Growth, Leech Seed,..."
3,Venusaur,"[Power Whip, Tackle, Growl, Vine Whip, Growth,..."
4,NaN,NaN
5,NaN,NaN
...,...,...
1021,NaN,NaN
1022,NaN,NaN
1023,NaN,NaN
1024,NaN,NaN


In [63]:
text = requests.get("https://bulbapedia.bulbagarden.net/wiki/Wigglytuff_(Pok%C3%A9mon)").text
soup = BeautifulSoup(text, "lxml")
data = soup.find_all("td", class_ = "roundy")

In [126]:
# ability and hidden ability
ability_data = data[2]
ability_data2 = ability_data.find_all(["a", "span"])
clear_ability = []
junk_text = ["Cacophony", "Abilities", "Gen IV+", "Battle Bond", "Battle Bond Greninja", "Gen VI+"]

for i in ability_data2:
    text = i.text.strip()
    if text and text not in junk_text:
        clear_ability.append(text)
        
if len(clear_ability) > 5:
    ability = clear_ability[0], clear_ability[2]
    hidden_ability = clear_ability[4]
elif len(clear_ability) < 5:
    ability = clear_ability[0]
    hidden_ability = clear_ability[2]
print(len(clear_ability), ability, hidden_ability)
clear_ability

6 ('Cute Charm', 'Competitive') Frisk


['Cute Charm', 'Cute Charm', 'Competitive', 'Competitive', 'Frisk', 'Frisk']